In [165]:
import pandas as pd
import numpy as np
import re

# Read original rFBA model

PDF file needs to be parsed manually...

(in order to be in accordance with the syntax of 'id' attribute values of the SBML type 'SId' (Reference: L3V2 Section 3.1.7), the metabolites 13PDG, 2PG, and 3PG were renamed to PDG13, PG2, and PG3, respectively.)

In [166]:
#import PyPDF2
#pdfFileObj = open('Covert2002/SI.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#pageObj = pdfReader.getPage(3)
#pageObj.extractText()

In [167]:
# reactions
SI1 = pd.read_csv('SI-1.tsv', sep='\t')

In [168]:
# metabolites
SI2 = pd.read_csv('SI-2.tsv', sep='\t')
SI2 = SI2.sort_values(by=['id'])

## Add Amino Acid-producing enzyme and Ribosome

In [169]:
SI1 = SI1.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Gene': 'aspA', 'Reaction': 'FUM <-> Z'}, ignore_index=True)
SI1 = SI1.append({'ID': 'R', 'Protein': 'Ribosome', 'Gene': 'rplA, rplB, rplC, rplD, rplE, rplF, rplI, rplJ, rplK, rplL, rplM, rplN, rplO, rplP, rplQ, rplR, rplS, rplT, rplU, rplV, rplW, rplX, rplY, rpmA, rpmB, rpmC, rpmD, rpmE, rpmF, rpmG, rpmH, rpmI, rpmJ, rpsA, rpsB, rpsC, rpsD, rpsE, rpsF, rpsG, rpsH, rpsI, rpsJ, rpsK, rpsL, rpsM, rpsN, rpsO, rpsP, rpsQ, rpsR, rpsS, rpsT, rpsU'}, ignore_index=True)

## Reformulate gene IDs

In [170]:
# Reformulation of "lumped" IDs
SI1.loc[SI1.ID == 'ACEE', 'Gene'] = 'aceE, aceF, lpdA'
SI1.loc[SI1.ID == 'ATPAR', 'Gene'] = 'atpA, atpB, atpC, atpD, atpE, atpF, atpG, atpH, atpI'
SI1.loc[SI1.ID == 'CYDA', 'Gene'] = 'cydA, cydB'
SI1.loc[SI1.ID == 'CYOA', 'Gene'] = 'cyoA, cyoB, cyoC, cyoD'
SI1.loc[SI1.ID == 'FDNG', 'Gene'] = 'fdnG, fdnH, fdnI'
SI1.loc[SI1.ID == 'FDOH', 'Gene'] = 'fdoI, fdoH, fdoG'
SI1.loc[SI1.ID == 'FRDA', 'Gene'] = 'frdA, frdB, frdC, frdD'
SI1.loc[SI1.ID == 'GLPA', 'Gene'] = 'glpA, glpB, glpC'
SI1.loc[SI1.ID == 'NUOA', 'Gene'] = 'nuoA, nuoB, nuoE, nuoF, nuoG, nuoH, nuoI, nuoJ, nuoK, nuoL, nuoM, nuoN'
SI1.loc[SI1.ID == 'PFLA', 'Gene'] = 'pflA, pflB'
SI1.loc[SI1.ID == 'PFLC', 'Gene'] = 'pflC, pflD'
SI1.loc[SI1.ID == 'PNTA1', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'PNTA2', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'SDHA1', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SDHA2', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SUCA', 'Gene'] = 'sucA, sucB, lpdA'
SI1.loc[SI1.ID == 'SUCCR', 'Gene'] = 'sucC, sucD'
SI1.loc[SI1.ID == 'GLCPTS', 'Gene'] = 'ptsG, ptsH, ptsI, crr'
SI1.loc[SI1.ID == 'PIUP2R', 'Gene'] = 'pitA, pitB'
SI1.loc[SI1.ID == 'RIBUPR', 'Gene'] = 'rbsA, rbsB, rbsC, rbsD'

In [171]:
# manual curation
SI1.loc[SI1.Protein == 'Catabolite activator protein', 'Gene'] = 'cra' # 'cra (fruR)' - beide Namen bezeichnen das gleiche Gen
SI1.loc[SI1.ID == 'GLCUP', 'Gene'] = 'galP' # 'galP, etc.' - galP ist ausreichend
SI1.loc[SI1.ID == 'PCKA', 'Gene'] = 'pck' # 'pckA'
SI1.loc[SI1.ID == 'SFCA', 'Gene'] = 'maeA' # 'sfcA'
SI1.loc[SI1.ID == 'GPMBR', 'Gene'] = 'gpmA' # 'sfcA' - gpmB gibt es nicht; gpmA katalysiert die gleiche Reaktion

In [172]:
genes_list = []

for index, value in SI1['Gene'].items():
    if isinstance(value, str):
        for g in value.split(', '):
            if g not in genes_list:
                genes_list.append(g)

# Species

In [173]:
# Split SI2 into intra- and extracellular metabolites

# add column with boolean variable
SI2['external'] = 1
SI2['external'] = SI2['name'].str.contains("External")

# split dataset
extracellular = SI2.query("external")
intracellular = SI2.query("external == False")

## Intracellular Metabolites $\mathcal{X}$

In [174]:
# drop boolean column
intracellular = intracellular.drop(['external'], axis=1)

In [175]:
# add amino acid
intracellular = intracellular.append({'id': 'Z', 'name': 'amino acid'}, ignore_index=True)

In [176]:
# add attributes
intracellular["compartment"] = 'cytosol'
intracellular["constant"] = 'false'
intracellular["boundaryCondition"] = 'false'
intracellular["hasOnlySubstanceUnits"] = 'true'
intracellular["ram:speciesType"] = 'metabolite'

## Extracellular Metabolites $\mathcal{Y}$

In [177]:
# drop boolean column
extracellular = extracellular.drop(['external'], axis=1)

In [178]:
# abundant species
extracellular.at[extracellular['id'] == 'CO2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'HEXT', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'PIxt', 'initialAmount'] = 10000

# potential product are initially zero
extracellular.at[extracellular['id'] == 'ETHxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'FORxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LCTSxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'PYRxt', 'initialAmount'] = 0

# carbon sources and O2: initial amount depends on the scenario; aerobic glucose as default:
extracellular.at[extracellular['id'] == 'ACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'GLCxt', 'initialAmount'] = 100
extracellular.at[extracellular['id'] == 'GLxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'O2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'RIBxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'SUCCxt', 'initialAmount'] = 0

In [179]:
# add attributes
extracellular["compartment"] = 'extracellular'
extracellular["constant"] = 'false'
extracellular["boundaryCondition"] = 'false' # not false for every species
extracellular["hasOnlySubstanceUnits"] = 'true'
extracellular["ram:speciesType"] = 'extracellular'

## Macromolecules $\mathcal{P}$

### Enzymes $\mathcal{E}$

In [180]:
# create copy of first part of SI
enzymes = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [181]:
# delete rows without ID (i.e. regulatory proteins), without Gene (i.e. spontaneous) or without Protein (exchange, biomass, maintenance)
enzymes.dropna(subset=['ID', 'Protein', 'Gene'], inplace=True) # in place: Keep df with valid entries in the same variable

In [182]:
# create enzyme IDs ('E_' + reaction id)
enzymes["id"] = 'E_' + enzymes["ID"]
enzymes = enzymes.drop(['ID'], axis=1) # delete old IDs
enzymes = enzymes[['id', 'Protein', 'Gene', 'Regulatory logic']] # order columns

In [183]:
enzymes = enzymes.rename(columns={"Protein": "name"})

In [184]:
# add attributes
enzymes["compartment"] = 'cytosol'
enzymes["constant"] = 'false'
enzymes["boundaryCondition"] = 'false'
enzymes["hasOnlySubstanceUnits"] = 'true'
enzymes["initialAmount"] = np.nan
enzymes["ram:molecularWeight"] = None # are set later
enzymes["ram:objectiveWeight"] = None # are set later
enzymes["ram:biomassPercentage"] = 0.0
enzymes["ram:speciesType"] = 'enzyme'
enzymes["degradationRate"] = 0.01

In [185]:
enzymes.at[enzymes["id"] == "R", "degradationRate"] = 0.001

### Regulatory Proteins $\mathcal{RP}$

In [186]:
# create copy of first part of SI
rp = SI1[['ID', 'Protein', 'Gene']].copy()

In [187]:
# keep rows without ID
rp = rp[pd.isnull(rp['ID'])]
rp = rp.drop(['ID'], axis=1)

In [188]:
# rp id = 'RP_' + gene id
rp["id"] = 'RP_' + rp["Gene"]
rp = rp[['id', 'Protein', 'Gene']] # order columns

In [189]:
rp = rp.rename(columns={"Protein": "name"})

In [190]:
# add attributes
rp["compartment"] = 'cytosol'
rp["constant"] = 'false'
rp["boundaryCondition"] = 'false'
rp["hasOnlySubstanceUnits"] = 'true'
rp["initialAmount"] = np.nan
rp["ram:biomassPercentage"] = 0.0
rp["ram:molecularWeight"] = None # are set later
rp["ram:objectiveWeight"] = None # are set later
rp["ram:speciesType"] = 'enzyme'
rp["degradationRate"] = 0.2

### Quota $\mathcal{Q}$

In [191]:
quota = SI1[['Reaction']].loc[SI1['Protein'] == "Biomass production flux"]

In [192]:
biomass_rxn = quota['Reaction'].values

In [193]:
stoich = []
spec = []
product = False

for s in biomass_rxn[0].split():
    if '-' in s:
        product = True
    try:
        s = float(s)
        if product:
            stoich.append(s)
        else:
            stoich.append(-s)
    except ValueError:
        if '+' not in s and '-' not in s:
            spec.append(s)

In [194]:
mw_mol = [503.15, 662.42, 741.38, 258.12, 258.12, 228.09, 198.07, 170.06, 183.03, 165.02, 87.05, 805.54, 130.06, 144.08, 763.50, 424.18, 94.98, 663.42, 740.38, 1000]
mw_mmol = [mw/1000 for mw in mw_mol]

In [195]:
biomass = pd.DataFrame({'id': spec, 'stoichiometry': stoich, 'molecular weight [g/mmol]': mw_mmol})

In [196]:
biomass['weight [g]'] = biomass['stoichiometry'] * biomass['molecular weight [g/mmol]']

In [197]:
# calculate weight of biomass
educt_surplus = biomass['weight [g]'].sum() - 1.0 # subtract biomass 'weight'
biomass_weight = -educt_surplus # new biomass weight equals educt surplus

In [198]:
# set weight of 'biomass', which will be reformulated as Quota_rest
biomass.at[biomass['id'] == 'Biomass', 'biomass_weight'] = biomass_weight

Fraction of proteins $\varphi_{\mathrm{protein}}$ is set to 0.7 (for details, see report), so the 'remaining' quota will make up 30 %

In [199]:
# add attributes for quota 'rest'
quota["ID"] = 'Quota_rest'
quota["ram:molecularWeight"] = biomass_weight
quota["ram:objectiveWeight"] = biomass_weight
quota["ram:biomassPercentage"] = 0.3
quota["degradationRate"] = 0.001

#### adjust protein fraction $\phi^*_{\text{protein}}$
Determine fraction of proteins explicitly modeled using data by *Li et al.* for E. coli on minimal medium

In [200]:
li = pd.read_csv('Li_minimal.tsv', sep='\t')

In [201]:
# add column specifying whether the protein is included in the model
li['included'] = 1
li['included'] = li['!Protein:Name'].isin(genes_list)

In [202]:
li_included = li.query("included")

In [203]:
n_missing_genes = len(genes_list)-len(li_included)
frac_explicit = li_included['!Abundance:[original]'].sum()/li['!Abundance:[original]'].sum()
frac_adjusted = 0.55*(1-li_included['!Abundance:[original]'].sum()/li['!Abundance:[original]'].sum())

In [204]:
print("%i genes from the model are not included in the dataset by Li et al." %n_missing_genes)

13 genes from the model are not included in the dataset by Li et al.


In [205]:
print("The fraction of protein explicitly modeled in the dataset by Li et al. is %f%%. Therefore, the adjusted protein quota is %f%% and the fraction of explicitly modeled proteins in the model is %f%%" %(frac_explicit*100, frac_adjusted*100, (0.55-frac_adjusted)*100))

The fraction of protein explicitly modeled in the dataset by Li et al. is 30.310474%. Therefore, the adjusted protein quota is 38.329239% and the fraction of explicitly modeled proteins in the model is 16.670761%


In [206]:
# add protein quota
# weights are determined later: length will be set to median length of all proteins explicitly modeled
quota = quota.append({"ID": 'Quota_protein', "Gene": 'R', "ram:biomassPercentage": frac_adjusted, "degradationRate": 0.01}, ignore_index=True)

In [207]:
quota['compartment'] = 'cytosol'
quota["initialAmount"] = np.nan

In [208]:
quota

,Reaction,ID,ram:molecularWeight,ram:objectiveWeight,ram:biomassPercentage,degradationRate,Gene,compartment,initialAmount
0,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...,Quota_rest,0.864801,0.864801,0.300000,0.001,NaN,cytosol,NaN
1,NaN,Quota_protein,NaN,NaN,0.383292,0.010,R,cytosol,NaN


# Reactions $\mathcal{R}$

## Metabolic Reactions $\mathcal{R_X}$ and Uptake Reactions $\mathcal{R_Y}$

In [209]:
# create copy of first part of SI
reactions = SI1[['ID', 'Protein', 'Gene', 'Reaction']].copy()

In [210]:
# delete rows without Reaction
reactions.dropna(subset=['Reaction'], inplace=True)

In [211]:
# delete exchange reactions
reactions = reactions[~reactions.Protein.str.contains('exchange')]

In [212]:
# delete maintenance and biomass reactions
reactions.drop(reactions[reactions.ID == 'ATPM'].index, inplace=True)
reactions.drop(reactions[reactions.ID == 'VGRO'].index, inplace=True)

In [213]:
# add reaction for amino acid
reactions = reactions.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Reaction': 'FUM <-> Z'}, ignore_index=True)

### turnover rates

In [214]:
reactions_turnover = pd.read_csv('TurnoverRates.tsv', sep='\t')

In [215]:
latex = reactions_turnover

In [216]:
latex = latex.drop(['Protein', 'Gene', 'reversible', 'Reaction'], axis=1) # delete old IDs

In [217]:
latex = latex[['Covert ID', 'BiGG ID', 'EC number', 'kcat_fwd/s', 'Organism (fwd)', 'kcat_bwd/s', 'Organism (bwd)']] # order columns

In [218]:
latex['latex'] = '\\\\'

In [219]:
latex.to_csv('turnover_latex.txt', sep='&', index=False)

In [220]:
# add column with boolean variable
reactions_turnover['transport'] = 1
reactions_turnover['transport'] = reactions_turnover['Protein'].str.contains("transport|efflux|uptake|secretion|permease")

# split dataset
turnover_transport = reactions_turnover.query("transport")
turnover_metabolic = reactions_turnover.query("transport == False")

In [221]:
missing = reactions_turnover['kcat_fwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_bwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_fwd/s'].str.count('diffusion').sum()+reactions_turnover['kcat_bwd/s'].str.count('diffusion').sum()

In [222]:
print("Total number of reactions and reverse reactions: %i" %(len(reactions_turnover)+reactions_turnover['reversible'].sum()))
print("Missing values (total): %i (%f %%)" %(missing, missing/(len(reactions_turnover)+reactions_turnover['reversible'].sum())*100))
print("Missing values (metabolic): %i (%f %%)" %(turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_metabolic)+turnover_metabolic['reversible'].sum())*100))
print("Missing values (transport): %i (%f %%)" %(missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_transport)+turnover_transport['reversible'].sum())*100))

Total number of reactions and reverse reactions: 152
Missing values (total): 37 (24.342105 %)
Missing values (metabolic): 7 (5.737705 %)
Missing values (transport): 30 (100.000000 %)


In [223]:
all_values = []

# Iterate over two given columns only from the dataframe
for column in reactions_turnover[['kcat_fwd/s', 'kcat_bwd/s']]:
    # Select column contents by column name using [] operator
    columnSeriesObj = reactions_turnover[column]
    for v in columnSeriesObj.values:
        try:
            v = float(v)
            if v > 0:
                all_values.append(v)
        except ValueError:
            pass

In [224]:
np.median(all_values)
#np.mean(all_values)

66.6

In [225]:
reactions_turnover = reactions_turnover.replace(to_replace = 'kcat', value = np.median(all_values))
reactions_turnover = reactions_turnover.replace(to_replace = 'diffusion', value = 0.0) # spontaneous rxns

In [226]:
reactions_turnover["kcat_fwd/s"] = pd.to_numeric(reactions_turnover["kcat_fwd/s"])
reactions_turnover["kcat_bwd/s"] = pd.to_numeric(reactions_turnover["kcat_bwd/s"])

In [227]:
reactions_turnover["fbc:geneProductAssociation"] = np.where(pd.isna(reactions_turnover["Gene"]), np.nan, 'E_' + reactions_turnover["Covert ID"])
reactions_turnover['kcat_fwd/min'] = reactions_turnover['kcat_fwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover['kcat_bwd/min'] = reactions_turnover['kcat_bwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover["fbc:maintenanceScaling"] = 0.0

## Translation $\mathcal{R_P}$

In [228]:
# Ribosomal activity: 17 amino acids / s
amino_acids_per_min = 1020
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
weight_amino_acid = 0.1331

In [229]:
translation = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [230]:
# drop rows without Gene (spontaneous reactions...)
translation.dropna(subset=['Gene'], inplace=True)

In [231]:
# rename old ID column to "Covert ID"
translation = translation.rename(columns={"ID": "Covert ID"})

In [232]:
# create new ID
translation["ID"] = np.where(pd.isna(translation["Covert ID"]), 'T_' + translation["Gene"], 'T_' + translation["Covert ID"])

In [233]:
translation["Length"] = None

In [234]:
uniprot = pd.read_csv('uniprot.tsv', sep='\t')

In [235]:
for index, value in translation['Gene'].items():
    length = 0
    for g in value.split(', '):
        try:
            length = length + uniprot.loc[uniprot['yourlist'] == g, 'Length'].values[0]
        except ValueError:
            pass
    translation['Length'][index] = length

In [236]:
median_length = translation['Length'].median()

In [237]:
# append translation of protein quota
translation = translation.append({"ID": 'T_Quota_protein', "Length": median_length}, ignore_index=True)

In [238]:
translation["reversible"] = 'false'
translation['ram:kcatForward'] = round(translation['Length'].apply(lambda x: amino_acids_per_min/x),5)
translation['ram:kcatBackward'] = 0.0
translation["fbc:maintenanceScaling"] = 0.0
translation["fbc:geneProductAssociation"] = 'R'

### set weights for macromolecules

In [239]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in enzymes['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('E_', 'T_'), 'Length'].values[0])
    enzymes.at[index, "ram:molecularWeight"] = weight
    enzymes.at[index, "ram:objectiveWeight"] = weight

In [240]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in rp['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('RP_', 'T_'), 'Length'].values[0])
    rp.at[index, "ram:molecularWeight"] = weight
    rp.at[index, "ram:objectiveWeight"] = weight

In [241]:
# now set weights of protein quota
quota.at[quota['ID'] == 'Quota_protein', 'ram:molecularWeight'] = median_length * weight_amino_acid
quota.at[quota['ID'] == 'Quota_protein', 'ram:objectiveWeight'] = median_length * weight_amino_acid

## Maintenance Reaction $\mathcal{R_M}$

In [242]:
# filter maintenance reaction from SI
main_df = SI1.filter(SI1[SI1.ID == 'ATPM'].index, axis = 0)

# Regulated Proteins $p = \mathcal{RE} \cup \mathcal{RP}$

In [243]:
# create copy of first part of SI
regulated = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [244]:
# delete rows without Regulation
regulated.dropna(subset=['Regulatory logic'], inplace=True)

In [245]:
# Create IDs of boolean variables representing gene expresssion for regulated proteins
# regulated enzymes: 'G_' + ID (uppercase)
# regulatory proteins: 'G_' + Gene (lowercase)
regulated["bool_ID"] = np.where(pd.isna(regulated["ID"]), 'G_' + regulated["Gene"] + '_bar', 'G_' + regulated["ID"] + '_bar')

## Events

In [246]:
events = pd.DataFrame()

### Control of boolean variables controlling RP gene expression

In [247]:
# effect of O2 only
events = events.append({'id':'low_O2', 'variable':'O2xt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_O2', 'variable':'O2xt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of SUCC only
events = events.append({'id':'low_SUCC', 'variable':'SUCCxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_SUCC', 'variable':'SUCCxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['1']}, ignore_index=True)
# effect of LCTS only
events = events.append({'id':'low_LCTS', 'variable':'LCTSxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_LCTS', 'variable':'LCTSxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of GL only
events = events.append({'id':'low_GL', 'variable':'GLxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL', 'variable':'GLxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['0']}, ignore_index=True)
# effect of GLC only
events = events.append({'id':'low_GLC', 'variable':'GLCxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GLC', 'variable':'GLCxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar'], 'value':['0']}, ignore_index=True)
# effect of RIB only
events = events.append({'id':'low_RIB', 'variable':'RIBxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_RIB', 'variable':'RIBxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP DcuS only
events = events.append({'id':'low_DcuS', 'variable':'RP_dcuS', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuS', 'variable':'RP_dcuS', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['1']}, ignore_index=True)
# effect of RP FadR only
events = events.append({'id':'low_FadR', 'variable':'RP_fadR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_FadR', 'variable':'RP_fadR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['1']}, ignore_index=True)

### complex regulation ###
# cra
# crp
# fadR
# pdhR

### Control of boolean variables controlling Enzyme gene expression

In [248]:
# effect of RP IclR only
events = events.append({'id':'low_IclR', 'variable':'RP_iclR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_IclR', 'variable':'RP_iclR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar'], 'value':['0']}, ignore_index=True)
# effect of RP PdhR only
events = events.append({'id':'low_PdhR', 'variable':'RP_pdhR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_PdhR', 'variable':'RP_pdhR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP FNR only
events = events.append({'id':'low_FNR', 'variable':'RP_FNR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar'], 'value':['0', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_FNR', 'variable':'RP_FNR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar'], 'value':['1', '1', '0']}, ignore_index=True)
# effect of RP ArcA only
events = events.append({'id':'low_arcA', 'variable':'RP_arcA', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_arcA', 'variable':'RP_arcA', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar'], 'value':['0']}, ignore_index=True)
# effects of Cra only
events = events.append({'id':'low_Cra', 'variable':'RP_cra', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar'], 'value':['0', '1']}, ignore_index=True)
events = events.append({'id':'high_Cra', 'variable':'RP_cra', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar'], 'value':['1', '0']}, ignore_index=True)
# effect of RpiR only
events = events.append({'id':'low_RpiR', 'variable':'RP_rpiR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RpiR', 'variable':'RP_rpiR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['0']}, ignore_index=True)

### complex regulation ###
# ACEB
# ADHER
# CYDA
# CYOA
# FRDA
# FUMAR, SDHA1, SDHA2
# PFLA, PFLC, FORUPR, DCUC

### regulation not modeled by Covert et al. ###
# FUMCR

## Qualitative Species

In [249]:
# initialize with gene names of regulatory proteins
qualspec = rp[['Gene']].copy()
qualspec['id'] = qualspec['Gene'] + '_bar'

In [250]:
# gene expression of regulated enzymes
for index, enzyme in enzymes['id'].items():
    if not pd.isna(enzymes['Regulatory logic'][index]):
        ID = enzyme[2:] + '_bar'
        qualspec = qualspec.append({'id': ID}, ignore_index=True)

In [251]:
qualspec['max'] = 1

# Parameters

In [252]:
parameters = {'epsilon_trans':0.000001, 'theta_xt':0.0001, 'theta_RP': 0.000001}

In [253]:
fbc_parameters = []

# Write Model as XML

In [254]:
rdefba = False

In [255]:
if rdefba:
    name = 'ecoli_rdeFBA'
else:
    name = 'ecoli_deFBA'

In [256]:
f = open(name + '.xml', 'w').close()
f = open(name + '.xml', 'w')

## First lines

In [257]:
# XML definition
f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")

39

In [258]:
# SBML definition
if rdefba:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:qual=\"http://www.sbml.org/sbml/level3/version1/qual/version1\" qual:required=\"true\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")
else:
    f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\">\n")


In [259]:
# Model Definition
f.write("<model id=\"" + name + "\" name=\"" + name + "\" fbc:strict=\"false\">\n\n")

64

## List of Compartments

In [260]:
f.write("<listOfCompartments>\n  <compartment id=\"extracellular\" constant=\"true\"/>\n  <compartment id=\"cytosol\" constant=\"true\"/>\n</listOfCompartments>\n\n")

142

## List of Species

In [261]:
f.write("<listOfSpecies>\n")

16

### Metabolites

#### extracellular metabolites $\mathcal{Y}$

In [262]:
for index, value in extracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+extracellular['name'][index]+"\" compartment=\"extracellular\" initialAmount=\""+str(extracellular['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\""+extracellular['boundaryCondition'][index]+"\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"extracellular\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### extracellular metabolites $\mathcal{X}$

In [263]:
for index, value in intracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+intracellular['name'][index]+"\" compartment=\"cytosol\" initialAmount=\"0.0\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"metabolite\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

### Macromolecules $\mathcal{P}$

#### Enzymes $\mathcal{E}$ (including Ribosome)

In [264]:
for index, value in enzymes['id'].items():
    f.write("<species id=\""+value+"\" name=\""+enzymes['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(enzymes['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(enzymes['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(enzymes['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### Regulatory Proteins $\mathcal{RP}$

In [265]:
if rdefba:
    for index, value in rp['id'].items():
        f.write("<species id=\""+value+"\" name=\""+rp['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(rp['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(rp['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(rp['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")

#### Quota $\mathcal{Q}$

In [266]:
for index, value in quota['ID'].items():
    f.write("<species id=\""+value+"\" compartment=\"cytosol\" initialAmount=\""+str(quota['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:species ram:molecularWeight=\""+str(quota['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(quota['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\""+str(quota['ram:biomassPercentage'][index])+"\" ram:speciesType=\"quota\"/>\n    </ram:RAM>\n  </annotation>\n</species>\n")    

In [267]:
f.write("</listOfSpecies>\n\n")

18

## List of Reactions $\mathcal{R}$

In [268]:
f.write("<listOfReactions>\n")

18

### Metabolic Reactions $\mathcal{R_X}$ and $\mathcal{R_Y}$

In [269]:
stoich = 1.0
stoich_bool = False
for index, value in reactions_turnover['Covert ID'].items():
    if '<' in reactions_turnover['Reaction'][index]: # reversible
        f.write("<reaction id=\""+value+"\" reversible=\"true\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/min'][index])+"\" ram:kcatBackward=\""+str(reactions_turnover['kcat_bwd/min'][index])+"\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    else:
        f.write("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/min'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    if pd.isna(reactions_turnover['fbc:geneProductAssociation'][index]): # spontaneous reactions
        pass
    else:
        f.write("  <fbc:geneProductAssociation fbc:id=\""+reactions_turnover['fbc:geneProductAssociation'][index]+"\">\n    <fbc:geneProductRef fbc:geneProduct=\""+reactions_turnover['fbc:geneProductAssociation'][index]+"\"/>\n  </fbc:geneProductAssociation>\n")
    f.write("  <listOfReactants>\n")
    for g in re.split(r'[+\s]\s*', reactions_turnover['Reaction'][index]):
        if '-' in g:
            f.write("  </listOfReactants>\n  <listOfProducts>\n")
        else:
            try:
                stoich = float(g)
                stoich_bool = True
            except ValueError:
                if g == "":
                    pass
                else:
                    if stoich_bool == False:
                        stoich = 1.0
                    f.write("    <speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                    stoich_bool = False
    f.write("  </listOfProducts>\n</reaction>\n")

### Translation Reactions $\mathcal{R_P}$

In [291]:
for index, value in translation['ID'].items():
    if translation['ID'][index] == 'T_Quota_protein':
        prefix = ''
    elif pd.notnull(translation['Covert ID'][index]):
        prefix = 'E_'
    # Regulatory Proteins have no "Covert ID"
    else:
        if rdefba:
            prefix = 'RP_'
        else:
            break
    if rdefba:
        if pd.notnull(translation['Regulatory logic'][index]):
            param_upper = value + '_FBCupper'
            param_lower = value + '_FBClower'
            fbc_parameters.append(param_upper)
            fbc_parameters.append(param_lower)
            f.write("<reaction id=\""+value+"\" reversible=\"false\" fbc:lowerFluxBound=\"" + param_lower + "\" fbc:upperFluxBound=\"" + param_upper + "\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        else:
            f.write("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
    else:
        print("<reaction id=\""+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
    print("  <fbc:geneProductAssociation fbc:id=\"E_R\">\n    <fbc:geneProductRef fbc:geneProduct=\"E_R\"/>\n  </fbc:geneProductAssociation>\n")
    print("  <listOfReactants>\n    <speciesReference species=\"Z\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"ATP\" stoichiometry=\""+str(3*translation['Length'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n")
    print("  <listOfProducts>\n    <speciesReference species=\""+prefix+translation['ID'][index][2:]+"\" stoichiometry=\"1.0\" constant=\"true\"/>\n    <speciesReference species=\"AMP\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"PPI\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"ADP\" stoichiometry=\""+str(2*translation['Length'][index])+"\" constant=\"true\"/>\n    <speciesReference species=\"PI\" stoichiometry=\""+str(2*translation['Length'][index])+"\" constant=\"true\"/>\n  </listOfProducts>\n</reaction>\n")   
    

<reaction id="T_ACEA" reversible="false">
  <annotation>
    <ram:RAM xmlns:ram="https://www.fairdomhub.org/sops/304">
      <ram:reaction ram:kcatForward="2.35023" ram:kcatBackward="0.0" ram:maintenanceScaling="0.0"/>
    </ram:RAM>
  </annotation>

  <fbc:geneProductAssociation fbc:id="E_R">
    <fbc:geneProductRef fbc:geneProduct="E_R"/>
  </fbc:geneProductAssociation>

  <listOfReactants>
    <speciesReference species="Z" stoichiometry="434" constant="true"/>
    <speciesReference species="ATP" stoichiometry="1302" constant="true"/>
  </listOfReactants>

  <listOfProducts>
    <speciesReference species="E_ACEA" stoichiometry="1.0" constant="true"/>
    <speciesReference species="AMP" stoichiometry="434" constant="true"/>
    <speciesReference species="PPI" stoichiometry="434" constant="true"/>
    <speciesReference species="ADP" stoichiometry="868" constant="true"/>
    <speciesReference species="PI" stoichiometry="868" constant="true"/>
  </listOfProducts>
</reaction>

<reaction i

### Quota_rest

In [271]:
f.write("<reaction id=\"R_Quota_rest\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")    
stoich = 1.0
stoich_bool = False
f.write("  <listOfReactants>\n")
for g in re.split(r'[+\s]\s*', quota['Reaction'][quota.index[quota['ID'] == 'Quota_rest'].tolist()[0]]):
    if '-' in g:
        f.write("  </listOfReactants>\n  <listOfProducts>\n")
    else:
        try:
            stoich = float(g)
            stoich_bool = True
        except ValueError:
            if g == "":
                pass
            else:
                if g == "Biomass":
                    g = "Quota_rest"
                    stoich = 1.0
                elif stoich_bool == False:
                    stoich = 1.0
                f.write("    <speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                stoich_bool = False
f.write("</listOfProducts>\n</reaction>\n")

30

### Degradation Reactions $\mathcal{R_{kd}}$

#### Enzymes

In [272]:
for index, value in enzymes['id'].items():
    f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(enzymes['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")

#### regulatory proteins

In [273]:
if rdefba:
    for index, value in rp['id'].items():
        f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
        f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(rp['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")    

#### Quota

In [274]:
for index, value in quota['ID'].items():
    f.write("<reaction id=\"kd_"+value+"\" reversible=\"false\">\n  <annotation>\n    <ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n      <ram:reaction ram:kcatForward=\"NaN\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n    </ram:RAM>\n  </annotation>\n")
    f.write("  <listOfReactants>\n    <speciesReference species=\""+value+"\" stoichiometry=\""+str(quota['degradationRate'][index])+"\" constant=\"true\"/>\n  </listOfReactants>\n</reaction>\n")    

### Maintenance Reaction

In [275]:
f.write("<reaction id=\"Maintenance\" name=\"Maintenance\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"0.14\"/>\n\t\t</ram:RAM>\n\t</annotation>\n\t<listOfReactants>\n\t\t<speciesReference species=\"ATP\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfReactants>\n\t<listOfProducts>\n\t\t<speciesReference species=\"ADP\" stoichiometry=\"1\" constant=\"true\"/>\n\t\t<speciesReference species=\"PI\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfProducts>\n</reaction>\n")

553

In [276]:
f.write("</listOfReactions>\n\n")

20

## List of Gene Products

### Enzymes (including Ribosome)

In [277]:
f.write("<fbc:listOfGeneProducts>\n")

25

In [278]:
for index, value in enzymes['id'].items():
    if enzymes['Gene'][index]:
        f.write("  <fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+enzymes['Gene'][index]+"_"+value[2:]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")
# Gene_Protein in order to have unique fbc:label values

### Regulatory Proteins

In [279]:
if rdefba:
    for index, value in rp['id'].items():
        f.write("  <fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+rp['Gene'][index]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")

In [280]:
f.write("</fbc:listOfGeneProducts>\n\n")

27

# List of Qualitative Species

In [281]:
if rdefba:
    f.write("<qual:listOfQualitativeSpecies>\n")
    for index, value in qualspec['id'].items():
        f.write("  <qual:qualitativeSpecies qual:id=\""+value+"\" qual:maxLevel=\""+str(qualspec['max'][index])+"\" qual:compartment=\"cytosol\" qual:constant=\"false\"/>\n")    
    f.write("</qual:listOfQualitativeSpecies>\n\n")

# List of Rules

In [282]:
if rdefba:
    f.write("<listOfRules>\n")

## AssignmentRules

### FBC rules

In [283]:
# translation always irreversible
if rdefba:
    for index, value in translation['ID'].items():
        if pd.notnull(translation['Regulatory logic'][index]):
            f.write("  <assignmentRule variable=\"" + value + "_FBCupper\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>NaN</ci>\n        <ci>qual_con_" + value[2:] + "_bar</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")
            f.write("  <assignmentRule variable=\"" + value + "_FBClower\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n      <apply>\n        <times/>\n        <ci>epsilon_trans</ci>\n        <ci>qual_con_" + value[2:] + "_bar</ci>\n      </apply>\n    </math>\n  </assignmentRule>\n")         
        

### Qual Rules

In [284]:
if rdefba:
    for qual in qualspec['id']:
        f.write("<assignmentRule variable=\"" + qual + "\">\n  <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n    <ci>qual_con_" + qual + "</ci>\n  </math>\n</assignmentRule>\n")

In [285]:
if rdefba:
    f.write("</listOfRules>\n\n")

# Events

In [286]:
if rdefba:
    f.write("<listOfEvents>\n")
    for index, value in events['id'].items():
        f.write("<event id=\"" + value + "\" useValuesFromTriggerTime=\"true\">\n  <trigger persistent=\"true\" initialValue=\"true\">\n    <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n")
        f.write("      <apply>\n        <"+events['relation'][index]+"/>\n        <ci>"+events['variable'][index]+"</ci>\n        <ci>"+events['threshold'][index]+"</ci>\n      </apply>\n    </math>\n  </trigger>\n  <listOfEventAssignments>\n")
        for i, assignment in enumerate(events['assignment'][index]):
            f.write("    <eventAssignment variable=\""+assignment+"\">\n      <math xmlns=\"http://www.w3.org/1998/Math/MathML\">\n        <cn type=\"integer\">"+events['value'][index][i]+"</cn>\n      </math>\n    </eventAssignment>\n")
        f.write("  </listOfEventAssignments>\n</event>\n")
    f.write("</listOfEvents>\n\n")

# List of Parameters

In [287]:
f.write("<listOfParameters>\n")
for index, value in parameters.items():
    f.write("  <parameter id=\"" + index + "\" constant=\"true\" value=\"" + str(value) + "\"/>\n")
if rdefba:
    for fbc_param in fbc_parameters:
        f.write("  <parameter id=\"" + fbc_param + "\" value=\"NaN\" constant=\"false\" />\n")
    # dummy variable for each qualitative species
    for qual in qualspec['id']:
        f.write("  <parameter id=\"qual_con_" + qual + "\" constant=\"false\" value=\"NaN\"/>\n")
f.write("</listOfParameters>\n")

20

## Last lines

In [288]:
f.write("</model>\n")
f.write("</sbml>")

7

In [289]:
f.close()